In [1]:
device = "cpu"
# device = "cuda"
import time,os,copy,torch,random_equivalent,torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from tqdm import tqdm
import pickle
# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(0)
np.random.seed(0)

# Plotting
import matplotlib.pyplot as plt


# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "16"

In [2]:
class_names = [
    "0",
    "1",
    "2",
    "3",
]
num_classes = len(class_names)

import idx2numpy 
import numpy as np 
arr3 = []
arr4 = []
#file1 = './MNIST/TorchvisionDatasetWrapper/raw/train-images-idx3-ubyte'
#file1 = './HybridQRNN/EMNIST/raw/gzip/emnist-digits-train-images-idx3-ubyte' 
#file1 = './HybridQRNN/KMNIST/raw/k49-train-imgs.npz' 
#arr = idx2numpy.convert_from_file(file1)
#arr = np.load(file1)['arr_0']
#file2 = './MNIST/TorchvisionDatasetWrapper/raw/train-labels-idx1-ubyte'
#file2 = './HybridQRNN/EMNIST/raw/gzip/emnist-digits-train-labels-idx1-ubyte' 
#file2 =  './HybridQRNN/KMNIST/raw/k49-train-labels.npz'
#arr2 = np.load(file2)['arr_0']
file1 = './MNIST/raw/train-images-idx3-ubyte' 
arr = idx2numpy.convert_from_file(file1)
file2 = './MNIST/raw/train-labels-idx1-ubyte' 
arr2 = idx2numpy.convert_from_file(file2)

for i in range(len(arr)):
    if arr2[i] in [0,1,2,3]:
        arr3.append(arr[i])
        arr4.append(arr2[i])


In [3]:
# repeat the images for n_steps
n_steps = 10
# train_sequences = np.tile(train_images[:, None], (1, n_steps, 1, 1))
# test_sequences = np.tile(test_images[:, None], (1, n_steps, 1, 1))
train_images = np.asarray(arr3,dtype=np.float32) / 255.0
train_labels = np.asarray(arr4,dtype=np.int64)

#train_labels -= 6
display(np.shape(train_labels))


(24754,)

In [4]:
# test_sequences = train_sequences[:int(len(train_sequences)*0.3)]
# test_labels = train_labels[:int(len(train_labels)*0.3)]
# train_sequences = train_sequences[int(len(train_sequences)*0.7):]
# train_labels = train_labels[int(len(train_labels)*0.7):]
split = np.array_split(train_images,[int(len(train_images)*0.7)])
train_sequences = split[0]
test_sequences = split[1]
print(len(split[0]),len(split[1]))
split = np.array_split(train_labels,[int(len(train_labels)*0.7)])
train_labels = split[0]
test_labels = split[1]
print(len(split[0]),len(split[1]))
#datalength = 5000
datalength = 100

train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))[:datalength]
#train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))
test_sequences = np.tile(test_sequences[:, None], (1, n_steps, 1, 1))
train_labels = train_labels[:datalength]
test_labels = test_labels

print(len(test_labels),len(test_sequences),len(train_labels),len(train_sequences))

17327 7427
17327 7427
7427 7427 100 100


In [ ]:
def train(input_model, train_x, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())
 
    input_model.train()
    for j in range(30):
        train_acc = 0
        loss_acc = 0

        for i in tqdm(range(train_x.shape[0] // minibatch_size)):
            input_model.zero_grad()
 
            batch_in = train_x[i * minibatch_size : (i + 1) * minibatch_size]
            # flatten images
            batch_in = batch_in.reshape((-1,) + train_x.shape[1:-2] + (784,))
            batch_label = train_labels[i * minibatch_size : (i + 1) * minibatch_size]
            output = input_model(torch.tensor(batch_in))
 
            # compute sparse categorical cross entropy loss
            logp = torch.nn.functional.log_softmax(output, dim=-1)
            logpy = torch.gather(logp, 1, torch.tensor(batch_label).view(-1, 1))
            loss = -logpy.mean()
            loss_acc += loss 
 
            loss.backward()
            optimizer.step()
 
            train_acc += torch.mean(
                torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
            )
        train_acc /= i + 1
        #torch.save(model_hybrid,f=f"./saves_class10/{n_qubits}x{q_depth}_{j}_{train_acc.numpy()}.h5")  
        print("Train accuracy (%d): " % j, train_acc.numpy())
        loss_acc /=i + 1
        print("Loss (%d): " % j, loss_acc.detach().numpy())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))
 
        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )
 
    test_acc /= i + 1
 
    print("Test accuracy:", test_acc.numpy())

In [ ]:
random_model = torch.nn.Sequential(
    torch.nn.Linear(784, 128),
    # set spiking_aware_training and a moderate dt
    random_equivalent.SpikingActivation(
    #Spike(
        torch.nn.ReLU(), dt=0.01, spiking_aware_training=True
    ),
    random_equivalent.TemporalAvgPool(),
    torch.nn.Linear(128, 10),
)

In [ ]:
train(random_model, train_sequences, test_sequences)

In [ ]:
torch.save(random_model,"./HybridQRNN/MNIST/random_mnist_final.h5")

In [5]:
random_model = torch.load("./HybridQRNN/MNIST/random_mnist_final.h5")

In [12]:
n_qubits = 6            # Number of qubits
q_depth = 2
q_delta = 0.01 
nqubits=n_qubits
tensor_length = n_qubits*(n_qubits-1)*q_depth+n_qubits

In [8]:
def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RZ_layer(w):
    for idx, element in enumerate(w):
        qml.RZ(element, wires=idx)

def entangling_layer(nqubits,weights):
    p = nqubits
    weights_ = (weight for weight in weights)
    for i in range(1,nqubits):
        for j in range(i):
            qml.CNOT(wires=[j,i])
            param = next(weights_)
            # vqc_params.append(param)
            qml.RZ(param, wires=i)
            p+=1
            param = next(weights_)
            # vqc_params.append(param)
            qml.RX(param, wires=i)
            p+=1
            qml.CNOT(wires=[j,i])

In [13]:
dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev, interface="torch")

def quantum_net(q_input_features, q_weights_flat):
    q_weights = q_weights_flat
    H_layer(n_qubits)
    RZ_layer(q_input_features)
    entangling_layer(nqubits,q_weights)
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)
    
class DressedQuantumNet(nn.Module):

    def __init__(self):

        super().__init__()
        self.pre_net = nn.Linear(128, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.randn(tensor_length))
        self.post_net = nn.Linear(n_qubits, 5)

    def forward(self, input_features):

        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))
            
        return self.post_net(q_out)



In [14]:
model_hybrid = copy.deepcopy(torch.nn.Sequential(*(list(random_model.children())[:-1])))
for param in model_hybrid.parameters():
    param.requires_grad = False

model_hybrid.fc = DressedQuantumNet()

model_hybrid = model_hybrid.to(device)

In [20]:
def train(input_model, train_x, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())
 
    input_model.train()
    for j in range(100):
        train_acc = 0
        loss_acc = 0
        
        for i in tqdm(range(train_x.shape[0] // minibatch_size)):
            input_model.zero_grad()
 
            batch_in = train_x[i * minibatch_size : (i + 1) * minibatch_size]

            batch_in = batch_in.reshape((-1,) + train_x.shape[1:-2] + (784,))
            batch_label = train_labels[i * minibatch_size : (i + 1) * minibatch_size]
            output = input_model(torch.tensor(batch_in))
 
            logp = torch.nn.functional.log_softmax(output, dim=-1)
            logpy = torch.gather(logp, 1, torch.tensor(batch_label).view(-1, 1))
            loss = -logpy.mean()
            loss_acc += loss 
 
            loss.backward()
            optimizer.step()
 
            train_acc += torch.mean(
                torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
            )
        train_acc /= i + 1  
        print("Train accuracy (%d): " % j, train_acc.numpy())
        metric_train=train_acc.numpy()

        loss_acc /=i + 1
        print("Loss (%d): " % j, loss_acc.detach().numpy())
        torch.save(model_hybrid,f="./model_saves/4_class/6x2_zz_{0}_{1}_{2}.h5".format(j+21,train_acc.numpy(),loss_acc.detach().numpy()))
        metric_loss= loss_acc.detach().numpy()
        #metric.append([metric_train,metric_loss])
        #np.save("metrics.npy",np.array(metric))
 
    # compute test accuracy
    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))
         
        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )
 
    test_acc /= i + 1
 
    print("Test accuracy:", test_acc.numpy())

In [16]:
def test(input_model, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())

In [ ]:
train(model_hybrid, train_sequences, test_sequences)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.74s/it]


Train accuracy (0):  0.23958333
Loss (0):  1.6340777


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:10<00:00,  3.36s/it]


Train accuracy (1):  0.23958333
Loss (1):  1.6280581


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.06s/it]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.69s/it]


Train accuracy (87):  1.0
Loss (87):  0.6176024


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.06s/it]


Train accuracy (88):  1.0
Loss (88):  0.610675


 33%|████████████████████████████████████▋                                                                         | 1/3 [00:03<00:07,  3.75s/it]